In [59]:
import os
import ssl
from pyesgf.logon import LogonManager
from pyesgf.search import SearchConnection

# local imports
from esgf_download.classes import Dataset

In [60]:
# Ensure that these environment variables are set, e.g. in your .bashrc
USERNAME = os.environ.get('ESGF_USERNAME')
PASSWORD = os.environ.get('ESGF_PASSWORD')

# login to ESGF and generate SSL context
myproxy_host = 'esgf-node.ipsl.upmc.fr'

lm = LogonManager()
lm.logon(username=USERNAME, password=PASSWORD, hostname=myproxy_host)

sslcontext = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
sslcontext.load_verify_locations(capath=lm.esgf_certs_dir)
sslcontext.load_cert_chain(lm.esgf_credentials)

# check if logged in
lm.is_logged_on()

True

In [ ]:
# define your query, eg:
query = {
    'project': 'CMIP6',
    'activity_drs': 'ScenarioMIP',
    'experiment_id': 'ssp534-over',
    'source_id': 'EC-Earth3-Veg',
    'variable': 'thetao',
    'frequency': 'mon',
    #'variant_label': 'r1i1p1f2',
    #'table_id': 'Omon',
}

# check ESGF for number of datasets that satisfy query
#search_node = 'http://esgf-data.dkrz.de/esg-search'
search_node = 'http://esgf-node.ipsl.upmc.fr/esg-search'
#search_node = 'https://esgf.ceda.ac.uk/esg-search'
conn = SearchConnection(search_node, distrib=True)
context = conn.new_context(**query, facets=query.keys())
context.hit_count

0

In [53]:
# generate results and check datasets to verify all is working as expected
results = context.search()

In [54]:
for result in results:
    ds = Dataset(result)
    if ds.end_date in ['230012', '229912']:
        print(ds.dataset_id)

CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp534-over.r1i1p1f1.Amon.tas.gr.v20221112|esg-dn1.nsc.liu.se


In [8]:
models = ['MRI-ESM2-0', 'CESM2-WACCM', 'CanESM5', 'IPSL-CM6A-LR', 'CNRM-ESM2-1']

In [29]:
print([file.start_date for file in ds.files])

['204001', '204101', '204201', '204301', '204401', '204501', '204601', '204701', '204801', '204901', '205001', '205101', '205201', '205301', '205401', '205501', '205601', '205701', '205801', '205901', '206001', '206101', '206201', '206301', '206401', '206501', '206601', '206701', '206801', '206901', '207001', '207101', '207201', '207301', '207401', '207501', '207601', '207701', '207801', '207901', '208001', '208101', '208201', '208301', '208401', '208501', '208601', '208701', '208801', '208901', '209001', '209101', '209201', '209301', '209401', '209501', '209601', '209701', '209801', '209901', '210001', '210101', '210201', '210301', '210401', '210501', '210601', '210701', '210801', '210901', '211001', '211101', '211201', '211301', '211401', '211501', '211601', '211701', '211801', '211901', '212001', '212101', '212201', '212301', '212401', '212501', '212601', '212701', '212801', '212901', '213001', '213101', '213201', '213301', '213401', '213501', '213601', '213701', '213801', '213901',

In [ ]:
test = {'a': 1, 'b': 2}
test2 = {'c': 3}

**test1

SyntaxError: can't use starred expression here (3259988686.py, line 4)